# Function ZNORM MIcro, Nano and Pico

### Import libraries

In [1]:
import pathlib
import pickle
import numpy as np
import pandas as pd
import oceans.sw_extras.sw_extras as swe
from itertools import chain

### Definition of paths

In [3]:
###############################################################################################
# # To run in home folder please uncomment below paths

Phys_MODEL_PATH = pathlib.Path("/home/jovyan/Phytoplankton_EOV/PFT_Product/Programs/Models/Phys_Models")
Micro_MODEL_PATH = pathlib.Path('/home/jovyan/Phytoplankton_EOV/PFT_Product/Programs/Models/Micro_Models')
Nano_MODEL_PATH = pathlib.Path('/home/jovyan/Phytoplankton_EOV/PFT_Product/Programs/Models/Nano_Models')
Pico_MODEL_PATH = pathlib.Path('/home/jovyan/Phytoplankton_EOV/PFT_Product/Programs/Models/Pico_Models')
###############################################################################################



###############################################################################################
# # To run in WorkSpace VRE folder please uncomment below paths

# Phys_MODEL_PATH = pathlib.Path("/workspace/VREFolders/Zoo-Phytoplankton_EOV/Phytoplankton_EOV/PFT_Product/Programs/Models/Phys_Models")
# Micro_MODEL_PATH = pathlib.Path('/workspace/VREFolders/Zoo-Phytoplankton_EOV/Phytoplankton_EOV/PFT_Product/Programs/Models/Micro_Models')
# Nano_MODEL_PATH = pathlib.Path('/workspace/VREFolders/Zoo-Phytoplankton_EOV/Phytoplankton_EOV/PFT_Product/Programs/Models/Nano_Models')
# Pico_MODEL_PATH = pathlib.Path('/workspace/VREFolders/Zoo-Phytoplankton_EOV/Phytoplankton_EOV/PFT_Product/Programs/Models/Pico_Models')
###############################################################################################

### Load the PCA model to transform temperature, salinity and spiciness profiles in principal components

In [4]:
# Lad the PCAs model of Temperature, Salinity and Spiciness
pkl_filename_temp_pca = ''.join([str(Phys_MODEL_PATH),str("/temp_pca_50_V1.pkl")])
pca_temp = pickle.load(open(pkl_filename_temp_pca, "rb"))

pkl_filename_sal_pca = ''.join([str(Phys_MODEL_PATH),str("/sal_pca_50_V1.pkl")])
pca_sal = pickle.load(open(pkl_filename_sal_pca, "rb"))

pkl_filename_spici_pca = ''.join([str(Phys_MODEL_PATH),str("/spici_pca_50_V1.pkl")])
pca_spici = pickle.load(open(pkl_filename_spici_pca, "rb"))


### Load the x-scaler and y-scaler models

In [5]:

# FOR MICRO
# Load the x-scaler from the file 
filename_x_scaler_Micro = ''.join([str(Micro_MODEL_PATH),str("/x_scaler_50_V1.pkl")]) 
x_scaler_Micro = pickle.load(open(filename_x_scaler_Micro, 'rb')) 
# Load the y-scaler from the file 
filename_y_scaler_Micro = ''.join([str(Micro_MODEL_PATH),str("/y_scaler_50_V1.pkl")])
y_scaler_Micro = pickle.load(open(filename_y_scaler_Micro, 'rb')) 


# FOR NANO
# Load the x-scaler from the file 
filename_x_scaler_Nano = ''.join([str(Nano_MODEL_PATH),str("/x_scaler_50_V1.pkl")]) 
x_scaler_Nano = pickle.load(open(filename_x_scaler_Nano, 'rb')) 
# Load the y-scaler from the file 
filename_y_scaler_Nano = ''.join([str(Nano_MODEL_PATH),str("/y_scaler_50_V1.pkl")])
y_scaler_Nano = pickle.load(open(filename_y_scaler_Nano, 'rb')) 



# FOR PICO
# Load the x-scaler from the file 
filename_x_scaler_Pico = ''.join([str(Pico_MODEL_PATH),str("/x_scaler_50_V1.pkl")]) 
x_scaler_Pico = pickle.load(open(filename_x_scaler_Pico, 'rb')) 
# Load the y-scaler from the file 
filename_y_scaler_Pico = ''.join([str(Pico_MODEL_PATH),str("/y_scaler_50_V1.pkl")])
y_scaler_Pico = pickle.load(open(filename_y_scaler_Pico, 'rb')) 




## Function SOCA_PFT_GLOBAL_2021 that takes inputs Rrs data, the SLA value, the PAR value, the vectors of salinity and temperature containing the 36 values for the 36 CMEMS depth levels

In [6]:
def INPUTS_SOCA_PFT_GLOBAL_2021(RHO_WN_412, RHO_WN_443, RHO_WN_490, RHO_WN_555, RHO_WN_670, SLA, PAR, MLD, sal,temp,spici,lon, lat, doy):
#                           sin_doy, cos_doy, x_cart, y_cart, z_cart, sal, temp, spici):
    
    # Transform in pandas dataframe
#     temp = pd.DataFrame(temp)
#     sal = pd.DataFrame(sal)
#     pres = pd.DataFrame(pres)
    INPUTS1 = [SLA, PAR, RHO_WN_412, RHO_WN_443, RHO_WN_490, RHO_WN_555, RHO_WN_670, MLD]
#               sin_doy,cos_doy,x_cart,y_cart,z_cart]
    
    # Compute spiciness from temperature and salinity
#     spiciness = swe.spice(sal, temp, pres)

    # Apply PCA on temperature, salinity and spiciness profiles to get the principal components
    PrincipalComponentsTemp_Test = pca_temp.transform(np.transpose(temp))[:,:4]
    principalComponentsSal_Test = pca_sal.transform(np.transpose(sal))[:,:3]
    principalComponentsSpici_Test = pca_spici.transform(np.transpose(spici))[:,:4]
    
    # Transform day of the year in sin and cos of the radians
    doy_radians = (doy*np.pi)/182.5
    DOY = [np.sin(doy_radians), np.cos(doy_radians)]
    
    # Transform the locationlon/lat in Cartesian coordinates x/y/z
    lat_radians = (lat*np.pi)/90
    lon_radians = (lon*np.pi)/180
    X_Y_Z = [np.cos(lat_radians) * np.cos(lon_radians), np.cos(lat_radians) * np.sin(lon_radians), np.sin(lat_radians)]
         
    INPUTS = INPUTS1 + DOY + X_Y_Z + np.concatenate(PrincipalComponentsTemp_Test).ravel().tolist() + np.concatenate(principalComponentsSal_Test).ravel().tolist() + np.concatenate(principalComponentsSpici_Test).ravel().tolist()
    
    input_features = ['sla','PAR',
                  'RHO_WN_412','RHO_WN_443','RHO_WN_490','RHO_WN_555','RHO_WN_670', 
                  'MLD','sin_doy', 'cos_doy',
                  'x_cart', 'y_cart', 'z_cart',
                  'temp_z_PC0', 'temp_z_PC1', 'temp_z_PC2', 'temp_z_PC3',
                  'sal_z_PC0', 'sal_z_PC1', 'sal_z_PC2',
                  'spici_z_PC0', 'spici_z_PC1', 'spici_z_PC2', 'spici_z_PC3',]



    INPUTS = pd.DataFrame(INPUTS)
    INPUTS = INPUTS.transpose()    
    INPUTS.columns = input_features
    INPUTS_scaled = pd.DataFrame(x_scaler_Micro.transform(INPUTS), columns=INPUTS.columns)
    
    return INPUTS_scaled

## FUNCTION SOCA_PFT_GLOBAL_2021

In [7]:
def SOCA_PFT_GLOBAL_2021(INPUTS_SOCA_PFT_GLOBAL_2021,ZNORM, pres_new):
    
    n_sample = INPUTS_SOCA_PFT_GLOBAL_2021.shape[0]
    
    YPRED_TOTAL_Micro = pd.DataFrame()
    YPRED_TOTAL_Nano = pd.DataFrame()
    YPRED_TOTAL_Pico = pd.DataFrame()
    
    nc1 = [95,95,97,98,99,101,103,105,107,108]
    nc2 = [63,71,77,58,90,77,94,104,102,80]
    Zeta_interp = np.linspace(0,1.5,50)
    depth_interp = Zeta_interp*ZNORM   
    
    for x in np.arange(len(nc1)):
        #######################################################################################################################################
         
        
        #######################################################################################################################################
        # FOR MICRO
        pkl_file_Micro = ''.join([str(Micro_MODEL_PATH),str("/pickle_model_Micro_"),str(x),str(".pkl")])
        MLP_Micro = pickle.load(open(pkl_file_Micro, 'rb')) 
        # Apply MLP to the test dataset
        ypred_scaled_Micro = MLP_Micro.predict(INPUTS_SOCA_PFT_GLOBAL_2021)        
        # Detransform the outputs:
        ypred_Micro = pd.DataFrame(y_scaler_Micro.inverse_transform(ypred_scaled_Micro), columns=np.arange(50), index=np.arange(n_sample))

#       Making vector
        Micro_PRED = ypred_Micro 
        Micro_PRED_vec = Micro_PRED.values.ravel()  
        Micro_PRED_vec = pow(10,Micro_PRED_vec)        
        Micro_PRED_vec_N = np.interp(pres_new, depth_interp,Micro_PRED_vec) # made into 36 depths
        Micro_PRED_vec_N = np.array(Micro_PRED_vec_N)
        Micro_PRED_vec_N[(pres_new > max(depth_interp))]=np.nan # masked data below 1.5 ZNORM depths      
        
#       Making matrix with 10 ensembles
        YPRED_TOTAL_Micro = pd.concat([YPRED_TOTAL_Micro, pd.DataFrame(Micro_PRED_vec_N)], axis=1)
    

        #######################################################################################################################################
        # FOR NANO
        pkl_file_Nano = ''.join([str(Nano_MODEL_PATH),str("/pickle_model_Nano_"),str(x),str(".pkl")])
        MLP_Nano = pickle.load(open(pkl_file_Nano, 'rb')) 
        # Apply MLP to the test dataset
        ypred_scaled_Nano = MLP_Nano.predict(INPUTS_SOCA_PFT_GLOBAL_2021)        
        # Detransform the outputs:
        ypred_Nano = pd.DataFrame(y_scaler_Nano.inverse_transform(ypred_scaled_Nano), columns=np.arange(50), index=np.arange(n_sample))

#       Making vector
        Nano_PRED = ypred_Nano 
        Nano_PRED_vec = Nano_PRED.values.ravel()  
        Nano_PRED_vec = pow(10,Nano_PRED_vec)        
        Nano_PRED_vec_N = np.interp(pres_new, depth_interp,Nano_PRED_vec) # made into 36 depths
        Nano_PRED_vec_N = np.array(Nano_PRED_vec_N)
        Nano_PRED_vec_N[(pres_new > max(depth_interp))]=np.nan # masked data below 1.5 ZNORM depths      
        
#       Making matrix with 10 ensembles
        YPRED_TOTAL_Nano = pd.concat([YPRED_TOTAL_Nano, pd.DataFrame(Nano_PRED_vec_N)], axis=1)
    
    
        #######################################################################################################################################
        # FOR PICO
        pkl_file_Pico = ''.join([str(Pico_MODEL_PATH),str("/pickle_model_Pico_"),str(x),str(".pkl")])
        MLP_Pico = pickle.load(open(pkl_file_Pico, 'rb')) 
        # Apply MLP to the test dataset
        ypred_scaled_Pico = MLP_Pico.predict(INPUTS_SOCA_PFT_GLOBAL_2021)        
        # Detransform the outputs:
        ypred_Pico = pd.DataFrame(y_scaler_Pico.inverse_transform(ypred_scaled_Pico), columns=np.arange(50), index=np.arange(n_sample))

#       Making vector
        Pico_PRED = ypred_Pico 
        Pico_PRED_vec = Pico_PRED.values.ravel()  
        Pico_PRED_vec = pow(10,Pico_PRED_vec)        
        Pico_PRED_vec_N = np.interp(pres_new, depth_interp,Pico_PRED_vec) # made into 36 depths
        Pico_PRED_vec_N = np.array(Pico_PRED_vec_N)
        Pico_PRED_vec_N[(pres_new > max(depth_interp))]=np.nan # masked data below 1.5 ZNORM depths      
        
#       Making matrix with 10 ensembles
        YPRED_TOTAL_Pico = pd.concat([YPRED_TOTAL_Pico, pd.DataFrame(Pico_PRED_vec_N)], axis=1) 
    
    
    
    #######################################################################################################################################
    #######################################################################################################################################
        
    soca_Micro=np.mean(YPRED_TOTAL_Micro, axis=1)
    soca_Micro_err=np.std(YPRED_TOTAL_Micro, axis=1)
    
    soca_Nano=np.mean(YPRED_TOTAL_Nano, axis=1)
    soca_Nano_err=np.std(YPRED_TOTAL_Nano, axis=1)
    
    soca_Pico=np.mean(YPRED_TOTAL_Pico, axis=1)
    soca_Pico_err=np.std(YPRED_TOTAL_Pico, axis=1)    
    
    
    return soca_Micro, soca_Micro_err, soca_Nano, soca_Nano_err, soca_Pico, soca_Pico_err